# State
First, define the state of the graph.
The state schema serves as the input schema for all nodes and edges in the graph.
Let's use the TypedDict class from python's typing module as our schema, which provides type hints for the keys

In [1]:
from typing import TypedDict

class State(TypedDict):
    graph_state: str

## Nodes
Nodes are just python functions. The first positional argument is the state, as defined above. Because the state is a TypedDict with schema as defined above, each node can access the key i.e. 'graph_state', with value i.e. state['graph_state']. Each node returns a new value of the state key 'graph_state'. By default the new value returned by each node will override the prior state value. 

In [2]:
def first_node(state: State):
    print("My first Node is called")
    return {"graph_state": state['graph_state'] + "I am playing"}

def second_node(state: State):
    print("My second Node is called")
    return {"graph_state": state['graph_state'] + " Cricket"}

def third_node(state: State):
    print("My third Node is called")
    return {"graph_state": state['graph_state'] + " Badminton"}

## Edges
Edges aconnect the nodes. Normal Edges are used if you want to always go from, for example, node_1 to node_2. Conditional Edges are used if you want to optionally route between nodes. Conditional edges are implemented as functionsthat return the nrxt node to visit based upon some logic.

In [8]:
import random
from typing import Literal

def decide_play(state: State) -> Literal['second_node', 'third_node']:
    graph_state = state['graph_state']

    if random.random() >= 0.5:
        return 'second_node'
    
    return 'third_node'


## Graph
### Graph Construction
Now, we build the graph from our components above. The StateGraph class is the graph class that we can use. First, we initialize a StateGraph with the state class we defined above. Then, add our nodes and edges. We use the START node, a special node that sends user input to the graph, to indicate where to start our graph. The END node is a special node that represents a terminal node. Finally, we compile our graph to perform a few basic checks on the graph structure. We can visualize the graph as a Mermaid diagram.

In [9]:
from IPython.display import Image, display
from langgraph.graph import START, END, StateGraph

# build the graph
builder = StateGraph(State)

builder.add_node("first_node", first_node)
builder.add_node("second_node", second_node)
builder.add_node("third_node", third_node)

## Logic
builder.add_edge(START, "first_node")
builder.add_conditional_edges("first_node", decide_play)
builder.add_edge("second_node", END)
builder.add_edge("third_node", END)

## Add
graph = builder.compile()

# View
#display(Image(graph.get_graph().draw_mermaid_png()))

### Graph invokation
The compiled graph implements runnable protocol. This provides a standard way to execute Langchain components, invoke is one of the standard methods in this interface. The input is a dictionary {"graph_state": "Hi this is lance"}, which sets the initial value for our graph state dict. When invoke is called, the graph starts execution from the START node. It progresses through the defined nodes (node_1, node_2,node_3) in order. The conditional edge will traverse from node_1 to node_2 or 3 using a 50/50 decision rule. Each node function receives the current state and returns a new value, which overrides the graph state. The executuon continues until it reaches the END node. 

In [24]:
graph.invoke({"graph_state": "Hi, My name is Kalyan,"})

My first Node is called
My second Node is called


{'graph_state': 'Hi, My name is Kalyan,I am playing Cricket'}

## Basic Chatbot

In [25]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [26]:
graph_builder = StateGraph(State)

In [28]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [29]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x117a74c20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x117a757c0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [30]:
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

In [31]:
graph_builder.add_node('chatbot', chatbot)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()


In [33]:
from IPython.display import Image, display

#display(Image(graph.get_graph().draw_mermaid_png()))

In [40]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

In [41]:
while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

User: What do you know about LangGraph?


InvalidUpdateError: Must write to at least one of ['graph_state']